<a href="https://colab.research.google.com/github/mostafa-ja/graph-tutorial/blob/master/Illegal_Bitcoin_Transactions2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying GCNLPA to Illicit Bitcoin Transaction Detection

## Outline

This notebook contains the code required to recreate the models and the (partial) results in the blog. In particular, it includes the following:
- Code to pre-process data and transofrm it into `torch_geometric.data.Data` format.
- LPA model implemented using PyTorch.
- GCN model implemented using PyTorch and PyTorch Geometric.
- GCNLPA model implemented using PyTorch and PyTorch Geometric.
- Code to train and perform hyperparamter optimization (using `optuna`) for each model.
- Code to evaluate the performance of the best trained model for each model.

## Setup

In [1]:

!pip install torch-geometric
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=06c58de6c4e8c2c51211432fb59d8f95f0641e9770460b12e7984bbada388452
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


In [2]:
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np 
import pandas as pd 
import torch 
import torch.nn.functional as F
import torch.optim as optim
import optuna
from optuna.trial import TrialState
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from google.colab import drive, files
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# 3.Install Kaggle API.
!pip install kaggle
# 4.Run the following code to configure the path to “kaggle.json”
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle datasets download -d ellipticco/elliptic-data-set
!unzip elliptic-data-set.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 98% 143M/146M [00:05<00:00, 34.7MB/s]
100% 146M/146M [00:05<00:00, 27.6MB/s]
Archive:  elliptic-data-set.zip
  inflating: elliptic_bitcoin_dataset/elliptic_txs_classes.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_features.csv  


In [ ]:
# set random seed
np.random.seed(42)
torch.manual_seed(42)

## Data

We allow this notebook to access the Elliptic Bitcoin Dataset. To do so, first download the dataset from this [link](https://www.kaggle.com/ellipticco/elliptic-data-set) and unzip the file. Then, upload the folder to Google Drive. Then, modify the `data_folder` variable in the cell below to the appropriate path to the uploaded folder. 

In [4]:
# specify path to folder on google drive that contains the data
data_folder = '/content'

# read in labels
classes = pd.read_csv(data_folder + "/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
# read in edge pairs
edges = pd.read_csv(data_folder + "/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
# read in features
features = pd.read_csv(data_folder + "/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

`classes` contains a tuple of node ID and the class of the node. Notice that some nodes have class `unknown`. We want to drop these nodes later.

In [5]:
classes.head(3)

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown


`edges` contains a pair of node IDs where $(v_1,v_2)\in$ `edges` means that there is an edge from node $v_1$ to node $v_2$.

In [ ]:
edges.head(3)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870


`features` contains a vector of 166 features for each node. Feature `0` represents the node ID and feature `1` represents the timestamp. Other columns are actual features of the nodes.

In [ ]:
features.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,-0.049707,-0.164402,-0.028741,-0.035391,-0.042955,-0.013282,-0.057195,-0.169609,-0.171154,-0.174473,-1.373657,-1.371460,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.120950,-0.199145,-0.187993,-0.212948,1.064205,1.063787,-1.373782,-1.354735,-0.297975,-1.403698,1.342003,1.340733,-0.171601,-0.458162,-0.423588,-0.440883,-1.015963,-1.01623,-0.968903,-0.375715,0.759748,-0.768329,1.488113,1.487932,-0.216814,-0.605631,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,-0.049707,-0.164417,-0.028741,-0.035391,-0.042955,-0.013282,-0.055327,-0.169757,-0.171477,-0.174490,0.887058,0.884557,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.121330,-0.110933,-0.075909,-0.111641,-1.159649,-1.160129,-1.373723,-1.353918,-0.295982,-1.403215,-0.975738,-0.975237,-0.168742,-0.263290,-0.186389,-0.250875,-1.015963,-1.01623,-0.968903,0.146997,1.366287,-0.464773,-1.116918,-1.116948,-0.216814,0.634272,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,-0.049707,-0.165054,-0.028741,-0.035391,-0.042955,-0.013282,-0.055298,-0.170400,-0.172217,-0.175227,0.887058,0.884557,-0.139729,-0.148911,-0.080147,-0.155660,-0.010763,-0.012107,-0.139731,-0.148906,-0.080147,-0.155660,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.129496,0.003143,0.002426,-0.122974,-0.041556,0.012549,-0.032244,-1.159649,-1.160129,-1.373902,-1.356210,-0.301548,-1.404577,-0.975738,-0.975237,-0.168742,-0.192468,-0.099790,-0.182133,-1.015963,-1.01623,-0.968903,-1.421138,-0.453330,-1.375441,-1.116918,-1.116948,-0.216814,0.407161,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792


We clean the data. We drop unclassified nodes because our goal is to perform supervised classification.

In [ ]:
# remap class, licit: 0, illicit: 1, unknown: -1
classes["class"] = classes["class"].map({"1": 1, "2": 0, "unknown": -1})

# merge features and labels
df = features.merge(classes, how="left", left_on=0, right_on="txId")
df = df.sort_values(0).reset_index(drop=True)
assert len(df) == len(classes)

# drop unclassified and isolated nodes
classified_nodes = set(classes[classes["class"] != -1]["txId"].values)
assert len(classified_nodes) == 46564
classified_edges = edges[(edges["txId1"].isin(classified_nodes)) & (edges["txId2"].isin(classified_nodes))].copy()
non_isolated_nodes = set(classified_edges["txId1"].values).union(classified_edges["txId2"].values)
classified_df = df[df[0].isin(non_isolated_nodes)].copy()

We extract a subset of the data as the data is too large to be run on Google Colab with the provisioned compute instance. The data consists of multiple connected graphs, where each graph corresponds to bitcoin transactions at a particular timestamp. Hence, the subsetting is performed by choosing a subset of timestamps. 

In [ ]:
# only use nodes with timestamp 0 - 21
# colab does not have enough memory to perform backpropagation when using all data
# DO NOT RUN THIS FOR ACTUAL EXPERIMENT
classified_nodes = set(classified_df[classified_df[1].between(0, 21)][0].values)
classified_edges = edges.loc[(edges["txId1"].isin(classified_nodes)) & (edges["txId2"].isin(classified_nodes))].copy()
non_isolated_nodes = set(classified_edges["txId1"].values).union(classified_edges["txId2"].values)
classified_df = df[df[0].isin(non_isolated_nodes)].copy()
print(len(non_isolated_nodes))

15310


We transform the data and store it as `torch_geometric.data.Data`. Realize that creating a `torch_geometric.data.Data` instance is straight forward: it suffices that we have tensors for the features, edge indices, and labels.

In [ ]:
# reindex nodes
classified_df = classified_df.sort_values(1).reset_index(drop=True)
old2new = {old:new for new, old in enumerate(classified_df[0].values)}
classified_edges["txId1"] = classified_edges["txId1"].map(old2new)
classified_edges["txId2"] = classified_edges["txId2"].map(old2new)
classified_df[0] = classified_df[0].map(old2new)

classified_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,txId,class
0,0,1,-0.172255,-0.184668,-1.201369,-0.046932,-0.043875,-0.02914,-0.061584,-0.163581,-0.168790,-0.044161,-0.165578,-2.516705,-2.486106,-0.042955,-0.013282,-0.053315,-0.170693,-0.172717,-0.175403,-1.373657,-1.37146,-0.139718,3.353530,4.932957,2.169850,-2.700912,-2.689592,-0.139720,3.353768,4.932954,2.170099,-2.700785,-2.689318,-0.024669,-0.031254,-0.022946,-0.026205,-6.996606,...,0.003143,0.002426,-0.124152,-0.081143,-0.037443,-0.077985,-1.159649,-1.160129,-1.373902,-1.218610,0.057412,-1.328036,-0.975738,-0.975237,-0.168742,0.896507,1.231774,0.874856,-1.015963,-1.01623,-0.968903,0.669709,1.972826,-0.161217,-1.116918,-1.116948,-0.193143,2.899243,3.697427,3.006855,-0.979074,-0.978556,-0.098889,-0.08749,-0.084674,-0.140597,1.519700,1.521399,204236566,0
1,1,1,-0.171546,-0.184668,-1.201369,-0.046932,-0.043875,-0.02914,-0.061584,-0.163640,-0.168016,-0.036565,-0.165215,-2.516705,-2.486106,-0.042955,-0.013282,-0.057328,-0.169678,-0.171222,-0.174563,-1.373657,-1.37146,-0.139732,-0.148912,-0.080147,-0.155662,-2.700912,-2.689592,-0.139734,-0.148907,-0.080146,-0.155662,-2.700785,-2.689318,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,0.003143,0.002426,-0.124499,-0.167579,-0.147106,-0.177400,-1.159649,-1.160129,-1.373693,-1.354814,-0.298387,-1.403698,1.342003,1.340733,-0.168742,-0.441084,-0.403790,-0.423447,-1.015963,-1.01623,-0.968903,0.146997,1.366287,-0.464773,-1.116918,-1.116948,-0.193143,-0.495145,-0.435113,-0.481158,-0.979074,-0.978556,0.018279,-0.08749,-0.131155,-0.097524,-0.120613,-0.119792,230528721,0
2,2,1,-0.172262,-0.184668,-1.201369,-0.046932,-0.043875,-0.02914,-0.061584,-0.163641,-0.168737,-0.043146,-0.165581,-2.516705,-2.486106,-0.042955,-0.013282,-0.057341,-0.170413,-0.172066,-0.175411,-1.373657,-1.37146,-0.139732,-0.148912,-0.080147,-0.155662,-2.700912,-2.689592,-0.139734,-0.148907,-0.080146,-0.155662,-2.700785,-2.689318,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-3.872194,-3.871849,-0.124848,-0.167579,-0.147027,-0.177472,-1.159649,-1.160129,-1.373693,-1.354814,-0.298387,-1.403698,1.342003,1.340733,-0.168742,-0.441084,-0.403790,-0.423447,-1.015963,-1.01623,-0.968903,0.146997,1.366287,-0.464773,-1.116918,-1.116948,-0.193143,-0.495145,-0.435113,-0.481158,-0.979074,-0.978556,0.018279,-0.08749,-0.131155,-0.097524,-0.120613,-0.119792,230528722,0


In [ ]:
# edges
edge_index = torch.tensor(classified_edges.values, dtype=torch.long)
edge_index = edge_index.t().contiguous()

# labels 
labels = classified_df["class"].values
labels = torch.tensor(labels, dtype=torch.float)

# timestamps 
timestamps = set(classified_df[1].values)

# features
features = torch.tensor(classified_df.drop([0, 1, "class", "txId"], axis=1).values, dtype=torch.float)

# construct torch_geometric.data.Data
data = Data(x=features, edge_index=edge_index, y=labels)

# visualize data
# this takes too long with the current size of the data
# g = torch_geometric.utils.to_networkx(data, to_undirected=False)
# plt.figure(figsize=(14,10))
# nx.draw(g, alpha=0.5, node_color=labels)

Lastly, we split the data into train/validation/test sets with 0.7/0.15/0.15 ratio. We use a stratified split because the labels are extremely imbalanced (there are far less fraudulant nodes than non-fraudulant nodes) and we want to make sure that each partition contains enough fraudulant nodes. Moreover, we do not split according to the timestamps. This is because nodes from different timestamps are disconnected, which means that if we split by timestamps, then we cannot "learn" edge weights for graphs in the validation/test sets during the training of GCNLPA.

In [ ]:
# generate array of indices
indices = np.arange(len(labels))

# split indices into train, val, and test sets
train_indices, test_indices, train_labels, test_labels = train_test_split(indices, labels, test_size=0.3, stratify=labels, random_state=42) 
val_indices, test_indices, val_labels, test_labels = train_test_split(test_indices, test_labels, test_size=0.5, stratify=test_labels, random_state=42) 

## GCN

### Model

This is the implementation of GCN using PyTorch and PyTorch Geometric. Our model consists of multiple layers of `GCNConv`, each of which is connected by a batch normalization and dropout components. The output is passed through a softmax layer being being returned to obtain propabilities. 

Notice that the implementation is relatively simple: we basically just have to import `GCNConv` from PyTorch Geometric and use it just like any other layer typically found in neural networks such as a linear layer. 

In [ ]:
# GCN
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GCN, self).__init__()
        
        convs = [GCNConv(input_dim, hidden_dim)] + [GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers-2)] + [GCNConv(hidden_dim, output_dim)]
        self.convs = torch.nn.ModuleList(convs)
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for _ in range(num_layers-1)])
        self.dropout = dropout
        self.softmax = torch.nn.Softmax(dim=1)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, data, adj_t=None):
        x, edge_index = data.x, data.edge_index
        for i, layer in enumerate(self.convs):
          x = layer(x, edge_index)
          if i < len(self.convs)-1:
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        out = self.softmax(x)
        return out

### Training and Testing

The below cell trains and evaluates the GCN model implemented above. Our procedure is as follows:
- Sample a set of hyperparameters from a predefined hyperparameter space.
- Train the model using the sampled set of hyperparamters using the train set.
- Evaluate the trained model using the validation set.
- Save the model paramters. 
- Choose the trained model with the best validation performance. Evaluate this model using the test set.

In [ ]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

def define_gcn(trial):
  n_layers = trial.suggest_int("n_layers", 2, 5)
  hidden_dim = trial.suggest_int("hidden_dim", 2**5, 2**8, log=True)
  dropout = trial.suggest_float("dropout", 0.3, 0.7)
  return GCN(165,hidden_dim,2,n_layers,dropout)
  
def objective_gcn(trial, data):
  model = define_gcn(trial).to(DEVICE)

  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  criterion = torch.nn.BCELoss()
  t = trial.suggest_float("t", 0.2, 0.6)

  for epoch in range(100):

    model.train()
    data = data.to(DEVICE)
    optimizer.zero_grad()
    out = model(data)

    tmp = torch.nn.functional.one_hot(data.y.type(torch.long)).type(torch.float)
    loss = criterion(out[train_indices], tmp[train_indices])
    y = out.detach()[:, 1]
    y = (y > t).type(torch.long)
    f1 = f1_score(data.y.cpu()[train_indices], y.cpu()[train_indices])
    
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
      valf1 = f1_score(data.y.cpu()[val_indices], y.cpu()[val_indices])
    trial.report(valf1, epoch)

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()
  
  torch.save(model.state_dict(), "gcn-" + str(trial.number) + ".pth")
  return valf1

def eval_gcn():

  # train and optimize hyperparameters
  study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
  study.optimize(
      lambda trial: objective_gcn(trial, data), n_trials=100, timeout=10000,
  )

  # result of hyperparamter optimization
  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  # retrieve best trial from hyperparameter optimization
  print("Best trial:")
  trial = study.best_trial
  print("  Value: ", trial.value)
  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
  print("\t Trial number: ", trial.number)

  # reconstruct best trained model
  state_dict = torch.load("gcn-" + str(trial.number) + ".pth")
  files.download("gcn-" + str(trial.number) + ".pth")
  model = GCN(165,trial.params["hidden_dim"],2,trial.params["n_layers"],trial.params["dropout"])
  model.load_state_dict(state_dict)

  # evaluate best trained model using test set
  model.to(DEVICE)
  model.eval()
  out = model(data)
  tmp = torch.nn.functional.one_hot(data.y.type(torch.long)).type(torch.float)
  y = out.detach()[:, 1]
  y = (y > trial.params["t"]).type(torch.long)
  f1 = f1_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  acc = accuracy_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  pre = precision_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  rec = recall_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  print("test performance:")
  print(f"\t f1: {f1}")
  print(f"\t acc: {acc}")
  print(f"\t pre: {pre}")
  print(f"\t rec: {rec}")

In [ ]:
eval_gcn()

[I 2021-12-08 07:33:24,655] A new study created in memory with name: no-name-75cb930a-dd48-49af-8684-0f6bba8409d9
[I 2021-12-08 07:33:37,433] Trial 0 finished with value: 0.13646055437100213 and parameters: {'n_layers': 3, 'hidden_dim': 231, 'dropout': 0.592797576724562, 'optimizer': 'Adam', 'lr': 0.00013066739238053285, 't': 0.546470458309974}. Best is trial 0 with value: 0.13646055437100213.
[I 2021-12-08 07:33:40,724] Trial 1 finished with value: 0.16411682892906815 and parameters: {'n_layers': 4, 'hidden_dim': 139, 'dropout': 0.308233797718321, 'optimizer': 'Adam', 'lr': 0.0002310201887845295, 't': 0.27336180394137355}. Best is trial 1 with value: 0.16411682892906815.
[I 2021-12-08 07:33:42,860] Trial 2 finished with value: 0.26499999999999996 and parameters: {'n_layers': 3, 'hidden_dim': 95, 'dropout': 0.4727780074568463, 'optimizer': 'RMSprop', 'lr': 0.0003839629299804173, 't': 0.3465447373174767}. Best is trial 2 with value: 0.26499999999999996.
[I 2021-12-08 07:33:45,548] Trial

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  84
  Number of complete trials:  16
Best trial:
  Value:  0.7528517110266161
  Params: 
    n_layers: 2
    hidden_dim: 99
    dropout: 0.41528211684678334
    optimizer: RMSprop
    lr: 0.001846906251904492
    t: 0.4300290296471279
	 Trial number:  37


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

test performance:
	 f1: 0.7449392712550608
	 acc: 0.9725729212015672
	 pre: 0.8518518518518519
	 rec: 0.6618705035971223


## GCNLPA

### Model

This is the implementation of GCNLPA. The GCN part is essentially the same as the earlier GCN model. The main differences here are that we have `edge_weight` as trainable parameters and add LPA in the `forward` function.

GCN and GCNLPA are non-trivially different. However, GCNLPA is still relatively straight forward to implement using PyTorch and PyTorch Geometric. 

In [ ]:
class GCNLPA(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, edge_dim, k):
        super(GCNLPA, self).__init__()
        
        convs = [GCNConv(input_dim, hidden_dim)] + [GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers-2)] + [GCNConv(hidden_dim, output_dim)]
        self.convs = torch.nn.ModuleList(convs)
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for _ in range(num_layers-1)])
        self.softmax = torch.nn.Softmax(dim=1)
        self.dropout = dropout
        self.edge_weight = torch.nn.Parameter(torch.ones(edge_dim))
        self.k = k

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, data, adj_t=None):
        # GCN
        x, edge_index = data.x, data.edge_index
        for i, layer in enumerate(self.convs):
          x = layer(x, edge_index, self.edge_weight.sigmoid())
          if i < len(self.convs)-1:
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        out = self.softmax(x)

        # LPA implementation with dense format
        labels = torch.nn.functional.one_hot(data.y.type(torch.long)).type(torch.float)
        matrix = torch_geometric.utils.to_dense_adj(data.edge_index, edge_attr=self.edge_weight.sigmoid(), max_num_nodes=data.num_nodes)
        matrix = matrix.squeeze(0)
        selfloop = torch.diag(torch.ones(matrix.shape[0])).to(DEVICE)
        matrix += selfloop
        for _ in range(self.k):
          y = torch.matmul(matrix, labels)
          labels = y
        return out, torch.nn.functional.normalize(labels, dim=1)

### Training and Testing

The below cell trains and evaluates the GCNLPA model implemented above. The procedure is identical to the earlier procedure.

In [ ]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

def define_gcnlpa(trial, edge_dim):
  n_layers = trial.suggest_int("n_layers", 2, 5)
  hidden_dim = trial.suggest_int("hidden_dim", 2**5, 2**8, log=True)
  dropout = trial.suggest_float("dropout", 0.3, 0.7)
  k = trial.suggest_int("k", 3, 7)
  return GCNLPA(165,hidden_dim,2,n_layers,dropout,edge_dim,k)
  
def objective_gcnlpa(trial, data):
  model = define_gcnlpa(trial, data.num_edges).to(DEVICE)

  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  criterion = torch.nn.BCELoss()
  criterion_lpa = torch.nn.BCELoss()
  t = trial.suggest_float("t", 0.2, 0.6)
  lambda_ = trial.suggest_int("lambda_", 1, 10)

  for epoch in range(100):

    model.train()
    data = data.to(DEVICE)
    optimizer.zero_grad()
    out, out_lpa = model(data)

    tmp = torch.nn.functional.one_hot(data.y.type(torch.long)).type(torch.float)
    loss = criterion(out[train_indices], tmp[train_indices])
    loss_lpa = criterion(out_lpa[train_indices], tmp[train_indices])
    loss += lambda_ * loss_lpa
    y = out.detach()[:, 1]
    y = (y > t).type(torch.long)
    f1 = f1_score(data.y.cpu()[train_indices], y.cpu()[train_indices])
    
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
      valf1 = f1_score(data.y.cpu()[val_indices], y.cpu()[val_indices])
    trial.report(valf1, epoch)

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()
  
  torch.save(model.state_dict(), "gcnlpa-" + str(trial.number) + ".pth")
  return valf1

def eval_gcnlpa():

  # train and optimize hyperparameters
  study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
  study.optimize(
      lambda trial: objective_gcnlpa(trial, data), n_trials=100, timeout=10000,
  )

  # result of hyperparameter optimization
  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  # retrieve best trial from hyperparameter optimization
  print("Best trial:")
  trial = study.best_trial
  print("  Value: ", trial.value)
  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
  print("\t Trial number: ", trial.number)

  # reconstruct best trained model
  state_dict = torch.load("gcnlpa-" + str(trial.number) + ".pth")
  files.download("gcnlpa-" + str(trial.number) + ".pth")
  model = GCNLPA(165,trial.params["hidden_dim"],2,trial.params["n_layers"],trial.params["dropout"],data.num_edges,trial.params["k"])
  model.load_state_dict(state_dict)

  # evaluate best trained model using test set
  model.to(DEVICE)
  model.eval()
  out, _ = model(data)
  tmp = torch.nn.functional.one_hot(data.y.type(torch.long)).type(torch.float)
  y = out.detach()[:, 1]
  y = (y > trial.params["t"]).type(torch.long)
  f1 = f1_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  acc = accuracy_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  pre = precision_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  rec = recall_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
  print("test performance:")
  print(f"\t f1: {f1}")
  print(f"\t acc: {acc}")
  print(f"\t pre: {pre}")
  print(f"\t rec: {rec}")

In [ ]:
eval_gcnlpa()

[I 2021-12-08 07:34:13,602] A new study created in memory with name: no-name-b6a40409-f436-41b9-8e72-b609b947953c
[I 2021-12-08 07:35:28,059] Trial 0 finished with value: 0.25641025641025644 and parameters: {'n_layers': 3, 'hidden_dim': 231, 'dropout': 0.592797576724562, 'k': 5, 'optimizer': 'Adam', 'lr': 0.005399484409787433, 't': 0.4404460046972835, 'lambda_': 8}. Best is trial 0 with value: 0.25641025641025644.
[I 2021-12-08 07:36:34,871] Trial 1 finished with value: 0.20416666666666666 and parameters: {'n_layers': 2, 'hidden_dim': 241, 'dropout': 0.6329770563201687, 'k': 4, 'optimizer': 'SGD', 'lr': 0.0011207606211860567, 't': 0.3727780074568463, 'lambda_': 3}. Best is trial 0 with value: 0.25641025641025644.
[I 2021-12-08 07:37:42,017] Trial 2 finished with value: 0.13872832369942195 and parameters: {'n_layers': 4, 'hidden_dim': 42, 'dropout': 0.41685785941408726, 'k': 4, 'optimizer': 'RMSprop', 'lr': 0.0010677482709481358, 't': 0.43696582754481694, 'lambda_': 1}. Best is trial 0 

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  90
  Number of complete trials:  10
Best trial:
  Value:  0.7601476014760148
  Params: 
    n_layers: 2
    hidden_dim: 89
    dropout: 0.31375540844608735
    k: 7
    optimizer: RMSprop
    lr: 0.001096821720752952
    t: 0.41868411173731185
    lambda_: 2
	 Trial number:  4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

test performance:
	 f1: 0.7795275590551182
	 acc: 0.9756203744013932
	 pre: 0.8608695652173913
	 rec: 0.7122302158273381


## LPA

### Model

In [ ]:
class LPA():
  def __init__(self, data, to_undirected = False):
    if data.is_directed() and to_undirected:
      edge_index = torch_geometric.utils.to_undirected(data.edge_index)
      data = Data(x=data.x, edge_index=edge_index, y=data.y)
    self.data = data
  
  def predict(self, k):
    A = torch_geometric.utils.to_dense_adj(self.data.edge_index, max_num_nodes=self.data.num_nodes).squeeze(0).to(DEVICE)
    selfloop = torch.diag(torch.ones(A.shape[0])).to(DEVICE)
    A += selfloop
    D = torch.diag(torch.sum(A, dim=1))
    D_inverse = torch.inverse(D).to(DEVICE)
    Y = self.data.y.clone().type(torch.LongTensor)
    Y[val_indices] = torch.zeros(len(val_indices)).type(torch.LongTensor)
    Y[test_indices] = torch.zeros(len(test_indices)).type(torch.LongTensor)
    Y = torch.nn.functional.one_hot(Y).type(torch.FloatTensor).to(DEVICE)
    for _ in range(k):
      Y_new = torch.matmul(torch.matmul(D_inverse, A), Y)
      Y_new[train_indices] = Y[train_indices]
      Y = Y_new
    Y = torch.argmax(Y, dim=1)
    self.prediction = Y[val_indices]
    self.target = self.data.y[val_indices]
    return Y 

### Training and Testing

In [ ]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

f1s = []
for k in range(3, 7):
  lpa = LPA(data, to_undirected=True)
  y = lpa.predict(k)
  f1 = f1_score(data.y.cpu()[val_indices], y.cpu()[val_indices])
  f1s.append(f1)
  print(k, f"\t f1: {f1}")

j = 3 + np.argmax(f1s)
y = lpa.predict(j)
f1 = f1_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
acc = accuracy_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
pre = precision_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
rec = recall_score(data.y.cpu()[test_indices], y.cpu()[test_indices])
print("test performance:")
print(f"\t f1: {f1}")
print(f"\t acc: {acc}")
print(f"\t pre: {pre}")
print(f"\t rec: {rec}")

3 	 f1: 0.3174603174603175
4 	 f1: 0.33507853403141363
5 	 f1: 0.37810945273631846
6 	 f1: 0.3762376237623763
test performance:
	 f1: 0.44976076555023925
	 acc: 0.9499346974314323
	 pre: 0.6714285714285714
	 rec: 0.3381294964028777


## Discussion

In this notebook, we walked through the process of implementing LPA, GCN, GCNLPA and applied them to the problem space of detecting fraudulent bitcoin. 
The model implementation is relatively straight forward using PyTorch and PyTorch Geometric. (The hyperparamter optimization can be easily performed using optuna as well.) 

We find that GCNLPA leads to slightly better performance compared to GCN in all four aspects of classification, which are arruracy, F1 score, precision, and recall. GCNLPA and GCN both significantly outperform LPA. 